# 🧹 Data Cleaning & Preprocessing — Practical Notebook

**Generated:** 2025-09-02 01:33 UTC · **Target stack:** pandas ≥ 2.x, NumPy, SciPy (optional)

This notebook is a hands‑on cheat sheet for the data cleaning & preprocessing phase:
- Creating a messy dataset
- Detecting & unifying missing values
- Fixing dtypes (numeric, date/time, categorical)
- String cleaning & regex parsing
- Handling duplicates
- Outliers (IQR, z‑score, winsorization)
- Imputation strategies (mean/median/mode, groupwise, ffill/bfill, interpolate)
- Category harmonization & binning
- Scaling & normalization (standard/min‑max/robust — manual)
- Date/time feature engineering
- Basic text cleaning
- Data validation checks
- A reusable `.pipe`‑based cleaning pipeline template

> ⚠️ **Version notes**
> * Avoid `DataFrame.append` (removed in pandas 2.0). Use `pd.concat` instead.
> * `DataFrame.applymap` deprecated in 2.1 — use `DataFrame.map` or vectorized ops.

---

## 0) Setup

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy import stats
import re

# Display options (feel free to tweak)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

RNG = np.random.default_rng(42)
pd.__version__

## 1) Create a deliberately messy dataset

In [ ]:
# Deliberately messy raw data
raw = pd.DataFrame({
    '  Customer Name  ': ['  Alice  ', 'Bob', 'Chloé', 'dAn', 'Eve', 'Bob', 'NA', None],
    'Age ': ['29', '  35', 'twenty', '42 ', None, '35', '27', '27'],
    'signup_date': ['2024-01-05', '05/02/2024', '2024/03/15', '15-04-2024', '2024-05-20', '2024-05-20', '', '2024-07-01'],
    'revenue($)': ['1,200.50', '$850', '€900', '1 100,00', '—', '850', '1200.50', 'null'],
    'city': ['New York', 'newyork', 'NY', 'San Francisco', 'SF', 'San-Francisco', 'n/a', 'New  York '],
    'phone': ['+1 (212) 555-0198', '212.555.0199', '212-555-0198', None, ' 212 555 0197 ', '2125550199', 'N/A', '212/555/0196'],
    'segment': ['Enterprise', 'SMB', 'smb', 'Mid-Market', 'Mid Market', 'SMB', 'enterprise', 'SMB'],
    'notes': ['Good client!!', ' late payer ', None, 'VIP;  high priority', 'Refund issued – check', '😊 Loyal', 'NA', '  ']
})
raw

## 2) Quick scan & profiling

In [ ]:
raw.head()

In [ ]:
raw.info()

In [ ]:
raw.describe(include='all', datetime_is_numeric=True)

In [ ]:
raw.isna().sum(), raw.eq('').sum(), raw.nunique()

## 3) Standardize column names (trim, snake_case, ASCII)

In [ ]:
import unicodedata

def normalize_colnames(cols):
    out = []
    for c in cols:
        c = c.strip()
        # Normalize unicode -> ASCII base
        c = unicodedata.normalize('NFKD', c).encode('ascii', 'ignore').decode('ascii')
        # Snake case-ish
        c = re.sub(r"[^0-9a-zA-Z]+", "_", c).strip('_').lower()
        out.append(c)
    return out

clean = raw.copy()
clean.columns = normalize_colnames(clean.columns)
clean.head()

## 4) Unify missing value tokens to `NaN`

In [ ]:
MISSING_TOKENS = {"na", "n/a", "null", "none", "—", "-", ""}

clean = clean.replace({col: {tok: np.nan for tok in MISSING_TOKENS} for col in clean.columns}, regex=False)
# Also strip whitespace in object cols
for c in clean.select_dtypes(include='object'):
    clean[c] = clean[c].str.strip()

clean.head()

## 5) Fix data types (numeric/date/categorical)

In [ ]:
# Age: coerce to numeric; invalid -> NaN
clean['age'] = pd.to_numeric(clean['age'], errors='coerce')

# Dates: try multiple formats robustly; coerce invalid to NaT
clean['signup_date'] = pd.to_datetime(clean['signup_date'], errors='coerce', dayfirst=True, infer_datetime_format=True)

# Revenue: remove currency/locale artifacts -> float
rev = (clean['revenue']
         .str.replace(r"[\s,]", "", regex=True)        # spaces & thousands separators
         .str.replace(r"^[\$€]", "", regex=True)        # leading currency symbol
         .str.replace(r"," , "", regex=False)           # leftover commas (safety)
      )
# Handle European comma decimal (e.g., 1 100,00 -> 1100.00 after removing spaces, then replace last comma->dot)
rev = rev.str.replace(r"(?P<num>\d+),(\d{2})$", lambda m: f"{m.group('num')}.{m.group(2)}", regex=True)
clean['revenue_usd'] = pd.to_numeric(rev, errors='coerce')

# Categorical candidates
for col in ['city', 'segment']:
    clean[col] = clean[col].astype('string').str.strip()

clean.dtypes

## 6) String cleaning & regex parsing (names, phones, categories)

In [ ]:
# Names: title case after trimming and unicode normalization
clean['customer_name'] = (clean['customer_name']
    .fillna('')
    .str.normalize('NFKD')
    .str.encode('ascii','ignore').str.decode('ascii')
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
)

# Phones: keep only digits; standardize as E.164-like for US (assume missing +1)
clean['phone_digits'] = (clean['phone']
    .astype('string')
    .str.replace(r"\D+", "", regex=True)
)
clean['phone_std'] = clean['phone_digits'].where(clean['phone_digits'].str.len()>0)
clean.loc[clean['phone_std'].str.len()==10, 'phone_std'] = '+1' + clean['phone_std']
clean.loc[clean['phone_std'].str.len()==11, 'phone_std'] = '+' + clean['phone_std']

# City harmonization
CITY_MAP = {
    'newyork':'New York', 'ny':'New York', 'new  york':'New York',
    'san francisco':'San Francisco', 'san-francisco':'San Francisco', 'sf':'San Francisco'
}
clean['city_norm'] = (clean['city']
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    .map(lambda x: CITY_MAP.get(x, x.title()) if isinstance(x, str) else x)
)

# Segment harmonization & categories
SEG_MAP = {'smb':'SMB', 'mid market':'Mid-Market', 'enterprise':'Enterprise'}
clean['segment_norm'] = (clean['segment']
    .str.lower().str.strip()
    .map(lambda x: SEG_MAP.get(x, x.title()))
    .astype('category')
)

clean[['customer_name','phone','phone_std','city','city_norm','segment','segment_norm']]

## 7) Detect & remove duplicates

In [ ]:
# Consider duplicates by name + phone_std + signup_date
dupe_mask = clean.duplicated(subset=['customer_name','phone_std','signup_date'], keep='first')
clean['is_duplicate'] = dupe_mask

clean_dupes_removed = clean.loc[~dupe_mask].copy()
clean_dupes_removed[['customer_name','phone_std','signup_date','is_duplicate']]

## 8) Outliers (IQR / z-score / winsorization)

In [ ]:
num = clean_dupes_removed[['age','revenue_usd']].copy()

# IQR method
iqr = num.quantile(0.75) - num.quantile(0.25)
lo = num.quantile(0.25) - 1.5 * iqr
hi = num.quantile(0.75) + 1.5 * iqr
bounds_iqr = pd.DataFrame({'low': lo, 'high': hi})

# z-score method
z = num.apply(lambda s: np.abs(stats.zscore(s, nan_policy='omit')))
zflag = (z > 3)

# Winsorization (cap extremes at 5th/95th percentiles)
wins = num.copy()
q05 = wins.quantile(0.05)
q95 = wins.quantile(0.95)
wins = wins.clip(lower=q05, upper=q95, axis=1)

bounds_iqr, zflag.sum(), wins.head()

## 9) Imputation strategies (mean/median/mode, ffill/bfill, interpolate, groupwise)

In [ ]:
impute = clean_dupes_removed.copy()

# Numeric global strategies
impute['age_mean'] = impute['age'].fillna(impute['age'].mean())
impute['age_median'] = impute['age'].fillna(impute['age'].median())

# Groupwise median by segment
impute['age_by_segment'] = impute.groupby('segment_norm')['age']                               .transform(lambda s: s.fillna(s.median()))

# Revenue interpolation (if time-indexed)
impute_sorted = impute.sort_values('signup_date').set_index('signup_date')
impute_sorted['revenue_interp'] = impute_sorted['revenue_usd'].interpolate(method='time')

# Categorical: fill with mode
mode_seg = impute['segment_norm'].mode(dropna=True)
if not mode_seg.empty:
    impute['segment_imputed'] = impute['segment_norm'].cat.add_categories(['_MISSING_'])
    impute['segment_imputed'] = impute['segment_imputed'].fillna(mode_seg.iloc[0])
else:
    impute['segment_imputed'] = impute['segment_norm']

# Forward/backward fill for phone
impute['phone_std_ffill'] = impute['phone_std'].ffill().bfill()

impute[['age','age_mean','age_median','age_by_segment','revenue_usd']].head(), impute_sorted[['revenue_usd','revenue_interp']].head()

## 10) Binning (equal width & quantile)

In [ ]:
# Create synthetic 'orders' to demonstrate
orders = pd.DataFrame({'amount': RNG.normal(100, 30, size=1000)})
orders['amount'] = orders['amount'].clip(lower=0)

orders['amount_bin_width'] = pd.cut(orders['amount'], bins=[0,50,100,150,200, np.inf], right=False)
orders['amount_bin_quantile'] = pd.qcut(orders['amount'], q=5)

orders['amount_bin_width'].value_counts().sort_index(), orders['amount_bin_quantile'].value_counts().sort_index()

## 11) Scaling & normalization (manual)

In [ ]:
scale_df = clean_dupes_removed[['age','revenue_usd']].astype(float)

# Standardization
standardized = (scale_df - scale_df.mean()) / scale_df.std(ddof=0)

# Min-Max scaling
minmax = (scale_df - scale_df.min()) / (scale_df.max() - scale_df.min())

# Robust scaling (IQR)
iqr = scale_df.quantile(0.75) - scale_df.quantile(0.25)
robust = (scale_df - scale_df.median()) / iqr.replace(0, np.nan)

standardized.head(), minmax.head(), robust.head()

## 12) Date/time features

In [ ]:
dt = clean_dupes_removed.copy()

# Ensure datetime
dt['signup_date'] = pd.to_datetime(dt['signup_date'], errors='coerce')

# Extract features
features = pd.DataFrame({
    'year': dt['signup_date'].dt.year,
    'month': dt['signup_date'].dt.month,
    'day': dt['signup_date'].dt.day,
    'dow': dt['signup_date'].dt.day_name(),
    'week': dt['signup_date'].dt.isocalendar().week.astype('Int64'),
    'is_month_start': dt['signup_date'].dt.is_month_start,
    'is_month_end': dt['signup_date'].dt.is_month_end,
})
features.head()

## 13) Basic text cleaning (punctuation, emoji, length)

In [ ]:
text = clean_dupes_removed[['notes']].copy()
text['notes'] = text['notes'].fillna('')

# Lowercase, trim, collapse spaces
text['notes_clean'] = (text['notes']
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Remove punctuation & emoji (basic heuristic)
text['notes_alpha'] = text['notes_clean'].str.replace(r"[^a-z0-9\s]", "", regex=True)

# Length features
text['char_len'] = text['notes_clean'].str.len()
text['word_count'] = text['notes_clean'].str.split().map(len)

text

## 14) Data validation checks (assertions & diagnostics)

In [ ]:
valid = clean_dupes_removed.copy()

errors = {}

# Example expectations
errors['age_nonnegative'] = valid.loc[~valid['age'].fillna(0).between(0, 120)]
errors['revenue_nonnegative'] = valid.loc[~valid['revenue_usd'].fillna(0).ge(0)]
errors['phone_format'] = valid.loc[~valid['phone_std'].fillna('').str.match(r"^\+\d{11}$|^$", na=False)]

# Primary key uniqueness (name+phone on same date)
pk_dupes = valid.duplicated(subset=['customer_name','phone_std','signup_date'], keep=False)
errors['primary_key_dupes'] = valid.loc[pk_dupes]

{k: v.shape[0] for k,v in errors.items()}, {k: v.head(3) for k,v in errors.items() if not v.empty}

## 15) Save cleaned data (CSV/Parquet) — optional

In [ ]:
# Uncomment to save
# clean_dupes_removed.to_csv('clean_customers.csv', index=False)
# clean_dupes_removed.to_parquet('clean_customers.parquet', index=False)

'Not saved (demo)'

## 16) Reusable cleaning pipeline template (.pipe)

In [ ]:
import unicodedata, re, numpy as np, pandas as pd

MISSING_TOKENS = {"na","n/a","null","none","—","-",""}
CITY_MAP = {
    'newyork':'New York', 'ny':'New York', 'new  york':'New York',
    'san francisco':'San Francisco', 'san-francisco':'San Francisco', 'sf':'San Francisco'
}
SEG_MAP = {'smb':'SMB', 'mid market':'Mid-Market', 'enterprise':'Enterprise'}


def step_standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    def norm(c):
        c = c.strip()
        c = unicodedata.normalize('NFKD', c).encode('ascii','ignore').decode('ascii')
        c = re.sub(r"[^0-9a-zA-Z]+", "_", c).strip('_').lower()
        return c
    df = df.copy()
    df.columns = [norm(c) for c in df.columns]
    return df


def step_unify_missing(df: pd.DataFrame) -> pd.DataFrame:
    repl = {col: {tok: np.nan for tok in MISSING_TOKENS} for col in df.columns}
    df = df.replace(repl, regex=False)
    for c in df.select_dtypes(include='object'):
        df[c] = df[c].str.strip()
    return df


def step_fix_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if 'age' in df:
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
    if 'signup_date' in df:
        df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce', dayfirst=True, infer_datetime_format=True)
    if 'revenue' in df or 'revenue($)' in df:
        col = 'revenue' if 'revenue' in df else 'revenue($)'
        rev = (df[col].astype('string')
                 .str.replace(r"[\s,]", "", regex=True)
                 .str.replace(r"^[\$€]", "", regex=True))
        rev = rev.str.replace(r"(?P<num>\d+),(\d{2})$", lambda m: f"{m.group('num')}.{m.group(2)}", regex=True)
        df['revenue_usd'] = pd.to_numeric(rev, errors='coerce')
    return df


def step_clean_strings(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if 'customer_name' in df:
        df['customer_name'] = (df['customer_name'].fillna('')
            .str.normalize('NFKD').str.encode('ascii','ignore').str.decode('ascii')
            .str.strip().str.replace(r"\s+"," ", regex=True).str.title())
    if 'phone' in df:
        phone = df['phone'].astype('string').str.replace(r"\D+", "", regex=True)
        std = phone.where(phone.str.len()>0)
        std = std.mask(std.str.len()==10, '+1' + std)
        std = std.mask(std.str.len()==11, '+' + std)
        df['phone_std'] = std
    if 'city' in df:
        df['city_norm'] = (df['city'].astype('string').str.lower().str.replace(r"\s+"," ", regex=True).str.strip()
            .map(lambda x: CITY_MAP.get(x, x.title()) if isinstance(x,str) else x))
    if 'segment' in df:
        df['segment_norm'] = (df['segment'].astype('string').str.lower().str.strip()
            .map(lambda x: SEG_MAP.get(x, x.title())).astype('category'))
    return df


def step_drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    key_cols = [c for c in ['customer_name','phone_std','signup_date'] if c in df]
    if key_cols:
        return df.drop_duplicates(subset=key_cols, keep='first')
    return df


def clean_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    return (df
        .pipe(step_standardize_columns)
        .pipe(step_unify_missing)
        .pipe(step_fix_dtypes)
        .pipe(step_clean_strings)
        .pipe(step_drop_duplicates)
    )

# Demo: apply to the original raw data
cleaned = clean_pipeline(raw)
cleaned.head()

---
## Wrap‑up
You now have a working notebook that demonstrates practical, reproducible data‑cleaning patterns using idiomatic pandas. Duplicate the functions in **Section 16** into your project and adapt the mapping dictionaries (cities, segments, etc.) to your domain.